# Polynomial Regression

## Announcements

The quiz tomorrow will have 3 questions.  The topics will be:
* Performing K-Means clustering using scikit-learn.  See the notebooks from Week 5.
* Performing K-Means clustering by hand (assign to clusters, find centroids, repeat).  We discussed this Wednesday of Week 5 (much of the discussion was at the whiteboard).  (Here is a [Wikipedia description](https://en.wikipedia.org/wiki/K-means_clustering#Standard_algorithm_(naive_k-means)) and an animation on [YouTube](https://youtu.be/5I3Ei69I40s).)
* Using a pandas DataFrame's `apply` method using a lambda function with `axis=0` (apply to one column at a time) or with `axis=1` (apply to one row at a time).  (We first introduced `apply` in [this notebook](https://christopherdavisuci.github.io/UCI-Math-10-S22/Week4/Week4-Friday.html#rescaling-2-using-apply), and it also was used in both worksheets last week.)

There isn't time to go over this material today, but Yasmeen will review the material on Tuesday. Yasmeen and I also have Zoom office hours the next two days:
* Yasmeen: 3:00-5:00pm Monday
* Chris: 11:00am-12:30pm Tuesday

See Canvas for the Zoom links to the office hours.

## Linear Regression with the cars dataset

Let's try to express "Horsepower" in terms of "MPG" (miles per gallon).  You should have the intuition that these two columns are negatively correlated.

In [1]:
import pandas as pd
import altair as alt

In [2]:
df = pd.read_csv("../data/cars.csv").dropna()
df.rename({"Miles_per_Gallon":"MPG"}, axis=1, inplace=True)

Looking at the following, we see that indeed, "MPG" and "Horsepower" are quite negatively correlated.

In [3]:
df.corr()

,MPG,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration
MPG,1.000000,-0.777618,-0.805127,-0.778427,-0.832244,0.423329
Cylinders,-0.777618,1.000000,0.950823,0.842983,0.897527,-0.504683
Displacement,-0.805127,0.950823,1.000000,0.897257,0.932994,-0.543800
Horsepower,-0.778427,0.842983,0.897257,1.000000,0.864538,-0.689196
Weight_in_lbs,-0.832244,0.897527,0.932994,0.864538,1.000000,-0.416839
Acceleration,0.423329,-0.504683,-0.543800,-0.689196,-0.416839,1.000000


This is reflected in the chart in the sense that as one coordinate value increases, the other tends to decrease.

In [5]:
alt.Chart(df).mark_circle().encode(
    x="MPG",
    y="Horsepower"
)

alt.Chart(...)

Let's investigate the same thing using scikit-learn.  We follow the usual pattern:
* import
* create/instantiate
* fit
* predict/transform

In [6]:
# import
from sklearn.linear_model import LinearRegression

In [7]:
# create/instantiate
reg = LinearRegression()

The following is probably the most common mistake with scikit-learn.  The first input needs to be two-dimensional (in this case, the first input needs to be a DataFrame, not a Series).

In [8]:
reg.fit(df["MPG"], df["Horsepower"])

ValueError: Expected 2D array, got 1D array instead:
array=[18.  15.  18.  16.  17.  15.  14.  14.  14.  15.  15.  14.  15.  14.
 24.  22.  18.  21.  27.  26.  25.  24.  25.  26.  21.  10.  10.  11.
  9.  27.  28.  25.  19.  16.  17.  19.  18.  14.  14.  14.  14.  12.
 13.  13.  18.  22.  19.  18.  23.  28.  30.  30.  31.  35.  27.  26.
 24.  25.  23.  20.  21.  13.  14.  15.  14.  17.  11.  13.  12.  13.
 19.  15.  13.  13.  14.  18.  22.  21.  26.  22.  28.  23.  28.  27.
 13.  14.  13.  14.  15.  12.  13.  13.  14.  13.  12.  13.  18.  16.
 18.  18.  23.  26.  11.  12.  13.  12.  18.  20.  21.  22.  18.  19.
 21.  26.  15.  16.  29.  24.  20.  19.  15.  24.  20.  11.  20.  19.
 15.  31.  26.  32.  25.  16.  16.  18.  16.  13.  14.  14.  14.  29.
 26.  26.  31.  32.  28.  24.  26.  24.  26.  31.  19.  18.  15.  15.
 16.  15.  16.  14.  17.  16.  15.  18.  21.  20.  13.  29.  23.  20.
 23.  24.  25.  24.  18.  29.  19.  23.  23.  22.  25.  33.  28.  25.
 25.  26.  27.  17.5 16.  15.5 

The key to fixing this error in this case is to use a list of column names (in this case, it's just the single column name "MPG", but we still need it to be in a list).

In [9]:
# fit
reg.fit(df[["MPG"]], df["Horsepower"])

LinearRegression()

In [10]:
reg.predict(df[["MPG"]])

array([125.3756586 , 136.8923227 , 125.3756586 , 133.05343467,
       129.21454664, 136.8923227 , 140.73121073, 140.73121073,
       140.73121073, 136.8923227 , 136.8923227 , 140.73121073,
       136.8923227 , 140.73121073, 102.34233041, 110.02010647,
       125.3756586 , 113.8589945 ,  90.82566631,  94.66455434,
        98.50344237, 102.34233041,  98.50344237,  94.66455434,
       113.8589945 , 156.08676286, 156.08676286, 152.24787483,
       159.9256509 ,  90.82566631,  86.98677828,  98.50344237,
       121.53677057, 133.05343467, 129.21454664, 121.53677057,
       125.3756586 , 140.73121073, 140.73121073, 140.73121073,
       140.73121073, 148.4089868 , 144.57009877, 144.57009877,
       125.3756586 , 110.02010647, 121.53677057, 125.3756586 ,
       106.18121844,  86.98677828,  79.30900221,  79.30900221,
        75.47011418,  60.11456205,  90.82566631,  94.66455434,
       102.34233041,  98.50344237, 106.18121844, 117.69788254,
       113.8589945 , 144.57009877, 140.73121073, 136.89

In [11]:
#add a column to df
df["Pred"] = reg.predict(df[["MPG"]])

Notice how the predicted values show up on the right side of the DataFrame.  And at least the first one is pretty accurate: the prediction is 125 whereas the true value is 130.

In [13]:
df.head()

,Name,MPG,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin,Pred
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA,125.375659
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA,136.892323
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA,125.375659
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA,133.053435
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA,129.214547


In [14]:
c = alt.Chart(df).mark_circle().encode(
    x="MPG",
    y="Horsepower"
)

We eventually want to plot the predicted values using a red line.  In the following chart `c1`, we are using the same "y" value as before, so the data is the exact same for now.

In [15]:
c1 = alt.Chart(df).mark_line(color="red").encode(
    x="MPG",
    y="Horsepower"
)

We can display two Altair charts in the same position, one "layered" over the other, using `+`.

In [16]:
c+c1

alt.LayerChart(...)

Now let's try the same thing, but using the "Pred" column for the y-values.

In [17]:
c1 = alt.Chart(df).mark_line(color="red").encode(
    x="MPG",
    y="Pred"
)

The following shows the line of best fit for this data.  (We will describe what is meant by "best" in a later class.)

In [19]:
c+c1

alt.LayerChart(...)

In [20]:
type(reg)

sklearn.linear_model._base.LinearRegression

We can find the slope of the line using `reg.coef_`.  The fact that this slope is negative corresponds to the fact that these variables are negatively correlated.

In [21]:
reg.coef_

array([-3.83888803])

The following is the y-intercept.  These sorts of stored parameter values are usually named in scikit-learn using a trailing underscore, so for example, using `intercept_` rather than the plain `intercept`.

In [23]:
reg.intercept_

194.47564319018676

Do you see how these values of -3.84 and 194.5 are reflected in the above line?

## Polynomial Regression with the cars dataset

Motto: Polynomial regression is no more difficult than linear regression.

We discussed at the whiteboard how finding coefficients in a polynomial can be viewed as a special case of linear regression.  (The reverse is also true, but here we are using the fact that we know how to find linear regression coefficients, to find polynomial coefficients.)

We just need to add some new columns to our DataFrame.  Here we use f-strings and a for-loop to keep our code Pythonic and DRY.

In [25]:
for i in range(1,4):
    df[f"MPG{i}"] = df["MPG"]**i

Notice how the true MPG value in the first row is 18, and so at the end we see the values $18$, $18^2$, and $18^3$.  This method would easily adapt to higher degrees.  (That is the main benefit of not just typing the three columns out one at a time.)

In [27]:
df.head()

,Name,MPG,Cylinders,Displacement,Horsepower,Weight_in_lbs,Acceleration,Year,Origin,Pred,MPG1,MPG2,MPG3
0,chevrolet chevelle malibu,18.0,8,307.0,130.0,3504,12.0,1970-01-01,USA,125.375659,18.0,324.0,5832.0
1,buick skylark 320,15.0,8,350.0,165.0,3693,11.5,1970-01-01,USA,136.892323,15.0,225.0,3375.0
2,plymouth satellite,18.0,8,318.0,150.0,3436,11.0,1970-01-01,USA,125.375659,18.0,324.0,5832.0
3,amc rebel sst,16.0,8,304.0,150.0,3433,12.0,1970-01-01,USA,133.053435,16.0,256.0,4096.0
4,ford torino,17.0,8,302.0,140.0,3449,10.5,1970-01-01,USA,129.214547,17.0,289.0,4913.0


In [28]:
reg = LinearRegression()

Here we make a list of the three columns we're interested in.

In [30]:
# cols = ["MPG1","MPG2","MPG3"]
# df.columns[-3:]
cols = [f"MPG{i}" for i in range(1,4)]

In [31]:
type(cols)

list

Because `cols` is already a list, it's important to use `df[cols]` instead of `df[[cols]]`.

In [32]:
reg.fit(df[cols],df["Horsepower"])

LinearRegression()

In [33]:
reg.coef_

array([-3.01312437e+01,  8.66793237e-01, -8.50566252e-03])

Here is a nice way to display which value corresponds to which coefficient.

In [34]:
pd.Series(reg.coef_, index=cols)

MPG1   -30.131244
MPG2     0.866793
MPG3    -0.008506
dtype: float64

In [35]:
reg.intercept_

429.5814606301147

The above values can be interpreted as saying that our model estimates the following formula, where H stands for horsepower and where M stands for miles per gallon.

$$
H \approx -0.0085 \cdot M^3 + 0.86 \cdot M^2 - 30.1 \cdot M + 429.6
$$

Let's add the values predicted by our model into the DataFrame.

In [36]:
df["Pred3"] = reg.predict(df[cols])

Let's get a new Altair chart.  We can save some typing by copying our linear chart `c1`.

In [37]:
c3 = c1.copy()

In [38]:
c3 = c3.encode(y="Pred3")

Warning: it often happens in Machine Learning that the better the fit appears to be, the worse it will perform on future data (due to *overfitting*).  So don't be too impressed by the accuracy of this fit.

In [40]:
c+c3

alt.LayerChart(...)

## Warning: Don't misuse polynomial regression

For some reason, unreasonable cubic models often get shared in the media.  The cubic polynomial that "fits best" can be interesting to look at, but don't expect it to provide accurate predictions in the future.  (This is a symptom of *overfitting*, which is a key concept in Machine Learning that we will return to soon.)

I don't know of any natural occurring phenomenon that follows a cubic pattern, so don't be deceived by the closeness of a cubic model fit.

### Example from FiveThirtyEight

Here a cubic model is being used to model the percentage of Yes confirmation votes as a function of time.

![cubic fit 1](../images/Cubic.png)

### Example from Trump's Covid-19 team

Using a cubic model to predict deaths from Covid-19.  This particular cubic model suggests deaths will get to 0 in mid-May 2020.

![cubic fit 2](../images/Cubic2.png)